<p style="font-family: times, serif; font-size:16pt; color:blue">Web scraping. Tipología y ciclo de vida de los datos aula 2- PRA1</p>
<div style="font-family: times, serif; font-size:12pt; text-align: right">Baltasar Boix y Yago Ezcurra, abril 2021</div>

In [1]:
import io
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

<p style="font-family: times, serif; font-size:12pt; color:blue">Descarga de la página Web del Portal oficial de turismo de España:
<a href="https://www.spain.info/es/consulta/monumentos-historicos-espana/">Monumentos de España</a></p>

In [2]:
def get_list_monumentos(link, params):
    res = requests.get(link, params=params)
    if res.status_code==200:
        print('success', res.url)
    else:
        print('fail', res.url)
    #print(res.cookies)
    x = BeautifulSoup(res.text).find_all('div', class_="wrapper-lista-resultados")[0]
    lst1 = [y.get_text() for y in x.find_all('p')]
    lst2 = [y['href'] for y in x.find_all('a')]
    return pd.DataFrame(list(zip(lst1, lst2)), columns =['Name', 'href'])

<p style="font-family: times, serif; font-size:12pt; color:blue">Bucle de lectura de todas las páginas</p>

In [3]:
df = get_list_monumentos('https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload', {})
for i in range(2, 1723//50+1):
    df = df.append(get_list_monumentos('https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload', 
                              {'page': i, 'rpp': 50}), ignore_index = True)
df.drop_duplicates(inplace=True)
df = df.reset_index(drop=True)

success https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload&page=1&rpp=50
success https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload&page=2&rpp=50
success https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload&page=3&rpp=50
success https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload&page=4&rpp=50
success https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload&page=5&rpp=50
success https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload&page=6&rpp=50
success https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload&page=7&rpp=50
success https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload&page=8&rpp=50
success https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload&page=9&rpp=50
success https://www.spain.info/es/consulta/monumentos-historicos-espana/?reload&page=10&rpp=50
success https://www.spain.info/es/consulta/monumentos-histo

In [4]:
df

,Name,href
0,Ermita de Nuestra Señora del Otero,/es/lugares-interes/ermita-nuestra-senora-otero/
1,Ermita de Nuestra Señora de la Piedad,/es/lugares-interes/ermita-nuestra-senora-piedad/
2,Ermita de Nuestra Señora del Rebollar,/es/lugares-interes/ermita-nuestra-senora-rebo...
3,Ermita de Los Remedios,/es/lugares-interes/ermita-remedios/
4,Ermita rupestre de Nuestra Señora de la Peña,/es/lugares-interes/ermita-rupestre-nuestra-se...
...,...,...
1594,Casa de L'Ardiaca,/es/lugares-interes/casa-ardiaca/
1595,Casa del Bailío,/es/lugares-interes/casa-bailio/
1596,Casa Batlló,/es/lugares-interes/casa-batllo/
1597,Casa Botines,/es/lugares-interes/casa-botines/


<p style="font-family: times, serif; font-size:12pt; color:blue">Lectura de datos de cada monumento obtenido en el paso anterior.</p>

In [5]:
def get_monumento(link):
    link = 'https://www.spain.info' + link
    with requests.Session() as s:
        res = s.get(link)
        s.cookies.clear()
    if res.status_code==200:
        print('success', res.url)
    else:
        print('fail', res.url)
    x0 = BeautifulSoup(res.text)
    x = x0.find_all('script')
    st = re.search(r'(?<=\{)([^}]+)(?=\})', str(x[2])).group(0)
    d = eval('{'+re.sub(' *\n|\t', '', st)+'}')
    if re.search(r'(?<=setView\()([^)]+)(?=\))', str(x[5])) is not None:
        st = re.search(r'(?<=setView\()([^)]+)(?=\))', str(x[5])).group(0)
        d['coordenadas'] = [re.search(r'(?<=\[)([^\]]+)(?=\])', st).group(0).split(',')]
    else:
        d['coordenadas'] = [['ND', 'ND']]
    del d['tipo-recurso']; del d['tipo-plantilla']; del d['zona-turistica']; del d['objetivo']
    del d['url-es']; del d['eje-primario']; del d['eje-secundario']; del d['category']
    if x0.find('section', class_="module entradilla").find('p', class_='text-secundario') != None:
        d['descripcion'] = re.sub(' *\n|\t', '', x0.find('section', 
                                                     class_="module entradilla").
                              find('p', class_='text-secundario').get_text())
    else:
        d['descripcion'] = ''
    x1 = x0.find_all('div', class_="title-box")
    d['direccion'] = [re.sub(' *\n|\t', '', x1[3].p.get_text()).split('\xa0')]
    #print(d)
    return pd.DataFrame.from_dict(d)

In [6]:
df1 = get_monumento(df.iloc[0,1])
for link in df.iloc[1:,1]:
    df1 = df1.append(get_monumento(link), ignore_index = True)

success https://www.spain.info/es/lugares-interes/ermita-nuestra-senora-otero/
success https://www.spain.info/es/lugares-interes/ermita-nuestra-senora-piedad/
success https://www.spain.info/es/lugares-interes/ermita-nuestra-senora-rebollar/
success https://www.spain.info/es/lugares-interes/ermita-remedios/
success https://www.spain.info/es/lugares-interes/ermita-rupestre-nuestra-senora-pena/
success https://www.spain.info/es/lugares-interes/ermita-san-adrian-sasabe/
success https://www.spain.info/es/lugares-interes/ermita-san-antonio-florida/
success https://www.spain.info/es/lugares-interes/ermita-san-babil-nuestra-senora-noravilla/
success https://www.spain.info/es/lugares-interes/ermita-san-baudelio-berlanga/
success https://www.spain.info/es/lugares-interes/ermita-san-blas/
success https://www.spain.info/es/lugares-interes/ermita-san-froilan/
success https://www.spain.info/es/lugares-interes/ermita-san-juan/
success https://www.spain.info/es/lugares-interes/ermita-san-julian-boada/

success https://www.spain.info/es/lugares-interes/iglesia-colegiata-santa-maria-mercado/
success https://www.spain.info/es/lugares-interes/iglesia-colegiata-santa-maria-vilabertran/
success https://www.spain.info/es/lugares-interes/iglesia-colegiata-santisimo-sacramento/
success https://www.spain.info/es/lugares-interes/iglesia-concatedral-santa-maria-asuncion/
success https://www.spain.info/es/lugares-interes/iglesia-concatedral-santa-maria-caceres/
success https://www.spain.info/es/lugares-interes/basilica-purissima-concepcio-barcelona/
success https://www.spain.info/es/lugares-interes/iglesia-purisima-concepcion-melilla/
success https://www.spain.info/es/lugares-interes/iglesia-nuestra-senora-concepcion-san-cristobal-laguna/
success https://www.spain.info/es/lugares-interes/iglesia-las-concepcionistas/
success https://www.spain.info/es/lugares-interes/iglesia-convento-san-esteban/
success https://www.spain.info/es/lugares-interes/iglesia-convento-trinitarias-descalzas/
success https

success https://www.spain.info/es/lugares-interes/arco-bara/
success https://www.spain.info/es/lugares-interes/arco-cuchilleros/
success https://www.spain.info/es/lugares-interes/arco-mazarelos/
success https://www.spain.info/es/lugares-interes/arco-romano/
success https://www.spain.info/es/lugares-interes/arco-ruinas-romanas-caparra-en-oliva-plasencia/
success https://www.spain.info/es/lugares-interes/arco-san-benito/
success https://www.spain.info/es/lugares-interes/arco-santa-maria/
success https://www.spain.info/es/lugares-interes/arco-trajano/
success https://www.spain.info/es/lugares-interes/arco-triunfo/
success https://www.spain.info/es/lugares-interes/area-funeraria-columbarios/
success https://www.spain.info/es/lugares-interes/los-arquillos-plaza-machete/
success https://www.spain.info/es/lugares-interes/arquitectura-mudejar-aragon/
success https://www.spain.info/es/lugares-interes/arte-rupestre-arco-mediterraneo-peninsular/
success https://www.spain.info/es/lugares-interes/a

success https://www.spain.info/es/lugares-interes/casa-palacio-valdes-salas/
success https://www.spain.info/es/lugares-interes/casa-pia-almoina/
success https://www.spain.info/es/lugares-interes/casa-los-picos/
success https://www.spain.info/es/lugares-interes/casa-pilatos/
success https://www.spain.info/es/lugares-interes/casa-rectoral-soto-luina/
success https://www.spain.info/es/lugares-interes/casa-salazar/
success https://www.spain.info/es/lugares-interes/casa-sol/
success https://www.spain.info/es/lugares-interes/casa-torre-los-lujanes/
success https://www.spain.info/es/lugares-interes/casa-torre/
success https://www.spain.info/es/lugares-interes/palacio-veletas/
success https://www.spain.info/es/lugares-interes/casa-vicens/
success https://www.spain.info/es/lugares-interes/casa-villa-casa-cisneros/
success https://www.spain.info/es/lugares-interes/casa-zafra/
success https://www.spain.info/es/lugares-interes/casas-blasonadas/
success https://www.spain.info/es/lugares-interes/cas

success https://www.spain.info/es/lugares-interes/castro-mohias/
success https://www.spain.info/es/lugares-interes/castro-monte-santa-tecla/
success https://www.spain.info/es/lugares-interes/castro-pineiro/
success https://www.spain.info/es/lugares-interes/castro-villamor/
success https://www.spain.info/es/lugares-interes/castros-ligonde/
success https://www.spain.info/es/lugares-interes/catedral-almeria/
success https://www.spain.info/es/lugares-interes/catedral-asuncion-nuestra-senora/
success https://www.spain.info/es/lugares-interes/catedral-avila/
success https://www.spain.info/es/lugares-interes/catedral-baeza/
success https://www.spain.info/es/lugares-interes/catedral-barbastro/
success https://www.spain.info/es/lugares-interes/catedral-barcelona/
success https://www.spain.info/es/lugares-interes/catedral-buen-pastor/
success https://www.spain.info/es/lugares-interes/catedral-el-burgo-osma/
success https://www.spain.info/es/lugares-interes/catedral-burgos/
success https://www.sp

...

success https://www.spain.info/es/lugares-interes/iglesia-santa-maria-salvador/
success https://www.spain.info/es/lugares-interes/iglesia-santa-maria-san-vicente-barquera/
success https://www.spain.info/es/lugares-interes/iglesia-santa-maria-santa-cruz-seros/
success https://www.spain.info/es/lugares-interes/iglesia-santa-maria-soto-luina/
success https://www.spain.info/es/lugares-interes/iglesia-santa-maria-tabara/
success https://www.spain.info/es/lugares-interes/iglesia-nuestra-senora-asuncion-tardajos/
success https://www.spain.info/es/lugares-interes/iglesia-santa-maria-taull/
success https://www.spain.info/es/lugares-interes/iglesia-santa-maria-tobed/
success https://www.spain.info/es/lugares-interes/basilica-santa-maria-uribarri/
success https://www.spain.info/es/lugares-interes/iglesia-santa-maria-asuncion-viana/
success https://www.spain.info/es/lugares-interes/iglesia-santa-maria-vizbayo/
success https://www.spain.info/es/lugares-interes/iglesia-santa-maria-wamba/
success htt

success https://www.spain.info/es/lugares-interes/termas-romanas-lugo/
success https://www.spain.info/es/lugares-interes/terrassa-parque-vall-paradis/
success https://www.spain.info/es/lugares-interes/torre-atalaya/
success https://www.spain.info/es/lugares-interes/torre-bajomedieval/
success https://www.spain.info/es/lugares-interes/torre-boabdil-recinto-amurallado/
success https://www.spain.info/es/lugares-interes/torre-bujaco/
success https://www.spain.info/es/lugares-interes/torre-calahorra/
success https://www.spain.info/es/lugares-interes/torre-canyamel/
success https://www.spain.info/es/lugares-interes/torre-caracol/
success https://www.spain.info/es/lugares-interes/torre-castrojeriz/
success https://www.spain.info/es/lugares-interes/torre-clavero/
success https://www.spain.info/es/lugares-interes/torre-cuadra/
success https://www.spain.info/es/lugares-interes/torreon-fernan-gonzalez/
success https://www.spain.info/es/lugares-interes/torre-dos-moreno/
success https://www.spain.i

success https://www.spain.info/es/lugares-interes/iglesia-salvador-cifuentes/
success https://www.spain.info/es/lugares-interes/iglesia-el-salvador-ejea-caballeros/
success https://www.spain.info/es/lugares-interes/iglesia-el-salvador-luesia/
success https://www.spain.info/es/lugares-interes/iglesia-salvador-requena/
success https://www.spain.info/es/lugares-interes/iglesia-salvador-santa-cruz-palma/
success https://www.spain.info/es/lugares-interes/iglesia-san-adriano/
success https://www.spain.info/es/lugares-interes/iglesia-san-andres-avila/
success https://www.spain.info/es/lugares-interes/iglesia-san-andres-cuenca/
success https://www.spain.info/es/lugares-interes/iglesia-san-andres-faba/
success https://www.spain.info/es/lugares-interes/iglesia-san-andres-olmedo/
success https://www.spain.info/es/lugares-interes/iglesia-san-andres-ponferrada/
success https://www.spain.info/es/lugares-interes/iglesia-san-andres-valverde-virgen/
success https://www.spain.info/es/lugares-interes/igl

success https://www.spain.info/es/lugares-interes/iglesia-villamor/
success https://www.spain.info/es/lugares-interes/iglesia-vilouriz/
success https://www.spain.info/es/lugares-interes/iglesia-virgen-angustias-alpajes/
success https://www.spain.info/es/lugares-interes/iglesia-virgen-luz/
success https://www.spain.info/es/lugares-interes/iglesia-romariz/
success https://www.spain.info/es/lugares-interes/juderia-belorado/
success https://www.spain.info/es/lugares-interes/juderia-cordoba/
success https://www.spain.info/es/lugares-interes/juego-oca/
success https://www.spain.info/es/lugares-interes/lonja-alcaniz/
success https://www.spain.info/es/lugares-interes/lonja/
success https://www.spain.info/es/lugares-interes/lonja-seda/
success https://www.spain.info/es/lugares-interes/lonja-zaragoza/
success https://www.spain.info/es/lugares-interes/palacio-polentinos/
success https://www.spain.info/es/lugares-interes/palacio-velada/
success https://www.spain.info/es/lugares-interes/palacio-ver

success https://www.spain.info/es/lugares-interes/palacio-miranda-valdecarzana/
success https://www.spain.info/es/lugares-interes/palacio-moja/
success https://www.spain.info/es/lugares-interes/palacio-momos/
success https://www.spain.info/es/lugares-interes/iglesia-parroquial-asuncion-nuestra-senora-espinosa/
success https://www.spain.info/es/lugares-interes/iglesia-parroquial-asuncion-nuestra-senora-viloria/
success https://www.spain.info/es/lugares-interes/iglesia-parroquial-asuncion-santa-colomba/
success https://www.spain.info/es/lugares-interes/iglesia-parroquial-galera/
success https://www.spain.info/es/lugares-interes/iglesia-parroquial-las-herrerias/
success https://www.spain.info/es/lugares-interes/iglesia-parroquial-magdalena/
success https://www.spain.info/es/lugares-interes/iglesia-parroquial-nuestra-senora-asuncion-arenas-san-pedro/
success https://www.spain.info/es/lugares-interes/iglesia-parroquial-nuestra-senora-asuncion-boadilla-camino/
success https://www.spain.info/

In [ ]:
df1.provincia = df1.provincia.str.strip()
df1.comunidad = df1.comunidad.str.strip()
df1.municipio = df1.municipio.str.strip()
df1.drop('detalle', axis=1, inplace=True)

In [15]:
df1[['latitude', 'longitude']] = pd.DataFrame(df1.coordenadas.to_list(), columns=['latitude', 'longitude'], index=df1.index)
df1.latitude = pd.to_numeric(df1.latitude, errors='coerce')
df1.longitude = pd.to_numeric(df1.longitude, errors='coerce')
df1.drop('coordenadas', axis=1, inplace=True)

<p style="font-family: times, serif; font-size:12pt; color:blue">Fusionamos los dataframes y generamos el fichero <b>monumentos_espana.csv</b>.</p>

In [16]:
df2 = pd.concat([df,df1], axis=1)

In [17]:
df2.to_csv('monumentos_espana.csv', index_label='id')